In [3]:
from logica import colab_logica


Could not import google.cloud.bigquery.
Could not import google.cloud.auth.
Could not import google.colab.widgets.


In [1]:
!echo 'Welcome to simple Souffle graph benchmarking.'
!echo 'Behold the power of The Machine:'
!echo "CPUs: $(nproc) / RAM: $(free -h | awk '/^Mem:/ {print $2}')"

Welcome to simple Souffle graph benchmarking.
Behold the power of The Machine:
CPUs: 32 / RAM: 125Gi


In [4]:
# Create the magic command
from IPython.core.magic import register_line_cell_magic
import duckdb

@register_line_cell_magic
def run_duckdb(line, cell=None):
    """
    Magic command for DuckDB queries.
    Usage:
      %run_duckdb SELECT 1 as x;
      %%run_duckdb
      SELECT * FROM range(10);
    """
    # Create or reuse connection
    if 'duckdb_conn' not in globals():
        global duckdb_conn
        duckdb_conn = duckdb.connect(':memory:')
        print('here!')

    query = cell if cell else line

    try:
        result = duckdb_conn.execute(query).fetchdf()
        return result
    except Exception as e:
        print(f"Error: {e}")
        return None


In [5]:
%%run_duckdb
SELECT value AS memlimit
FROM duckdb_settings()
WHERE name = 'memory_limit' or name = 'threads';

here!


,memlimit
0,100.6 GiB
1,32


In [14]:
%%run_duckdb
INSTALL duckpgq FROM community;
LOAD duckpgq;


,Success


In [15]:
from IPython.core.magic import register_cell_magic
from IPython import get_ipython
import time
from logica.common import sqlite3_logica
import pandas

timing = {}

reports = []

@register_cell_magic
def loop(line, cell):
    global timing
    local_timing = {}
    ip = get_ipython()
    # Evaluate the line to get the list (e.g., "my_files")
    problem_name, iterator = ip.ev(line) 
    
    for item in iterator:
        # Inject 'item' into global namespace so the inner magic sees it
        print('Running %s.' % item)
        ip.user_ns['loop_parameter'] = item 
        # Run the content as a new cell execution
        start_time = time.perf_counter()
        ip.run_cell(cell.replace('{loop_parameter}', item))
        end_time = time.perf_counter()
        elapsed = end_time - start_time
        timing[item] = elapsed
        local_timing[item] = elapsed
    report = (' === Timing for %s === // Cypher //\n' % problem_name) + (
        sqlite3_logica.DataframeAsArtisticTable(pandas.DataFrame({'problem': list(local_timing.keys()),
                                                                  'time': list(local_timing.values())})))
    reports.append(report)
    print(report)
    

In [28]:
from IPython.core.magic import register_cell_magic
from IPython import get_ipython
import time
from logica.common import sqlite3_logica
import pandas
import signal

timing = {}
reports = []

@register_cell_magic
def loop(line, cell):
    global timing
    local_timing = {}
    ip = get_ipython()
    
    problem_name, iterator = ip.ev(line) 
    
    # Kill switch state
    stop_execution = False 
    
    # 1. Define a container to hold the timeout status
    # We use a list/dict so the inner function can mutate it (closure scope hack)
    status = {'timed_out': False}

    # 2. The Handler: Sets the flag AND raises hell
    def timeout_handler(signum, frame):
        status['timed_out'] = True
        print('Timeout detected.')
        # We raise KeyboardInterrupt because DuckDB respects it better than custom exceptions
        raise KeyboardInterrupt("Time Limit Exceeded")

    for item in iterator:
        print('Running %s.' % item)
        ip.user_ns['loop_parameter'] = item 
        
        # SKIP LOGIC
        if stop_execution:
            print(f"Skipping {item} (Previous timeout).")
            timing[item] = 'TIMEOUT (>200s)'
            local_timing[item] = 'TIMEOUT (>200s)'
            continue

        start_time = time.perf_counter()
        
        # Reset flag for this run
        status['timed_out'] = False
        
        # Save original handler to restore later
        original_handler = signal.getsignal(signal.SIGALRM)
        
        # Install our handler
        signal.signal(signal.SIGALRM, timeout_handler)
        signal.alarm(200) # <--- TIMEOUT SETTING
        
        try:
            # Run the cell
            result = ip.run_cell(cell.replace('{loop_parameter}', item))
        finally:
            # DISABLE ALARM IMMEDIATELY
            signal.alarm(0)
            # Restore original handler
            signal.signal(signal.SIGALRM, original_handler)

        end_time = time.perf_counter()
        
        # 3. CHECK THE FLAG
        # We don't care what result.error_in_exec is. If the flag is up, we timed out.
        if status['timed_out']:
            print(f"!!! TIMEOUT detected on {item} !!!")
            stop_execution = True
            elapsed = 'TIMEOUT (>200s)'
        elif result.error_in_exec:
            # It failed for some other reason (Syntax error, etc)
            # We treat this as a crash, but not necessarily a timeout chain-reaction
            print(f"Error executing {item}: {result.error_in_exec}")
            elapsed = 'ERROR' 
        else:
            elapsed = end_time - start_time
        
        timing[item] = elapsed
        local_timing[item] = elapsed

    report = (' === Timing for %s === // Cypher //\n' % problem_name) + (
        sqlite3_logica.DataframeAsArtisticTable(pandas.DataFrame({'problem': list(local_timing.keys()),
                                                                  'time': list(local_timing.values())})))
    reports.append(report)
    print(report)

In [29]:
%%run_duckdb
SELECT version();


,"""version""()"
0,v1.3.2


In [30]:
%%run_duckdb
-- Step 1: Connect to the material world (The Database)
USE memory;
DETACH DATABASE IF EXISTS graphs;
ATTACH 'graphs.db' as graphs;
USE graphs;

,Success


In [31]:
graphs = ['G%dk' % i for i in [1,2,3,4,5]]
#graphs = ['G5c', 'G6c', 'G7c']

In [32]:
%%run_duckdb
INSTALL duckpgq FROM community; 
LOAD duckpgq; 

,Success


In [33]:
%%loop ('Transitive Closure', graphs)
%%run_duckdb

-- Step 2: Ensure the magic is loaded
LOAD duckpgq;

-- Step 3: Materialize the Vertices
CREATE OR REPLACE TABLE GXk_nodes AS 
    SELECT DISTINCT col0 AS id FROM {loop_parameter} 
    UNION 
    SELECT DISTINCT col1 AS id FROM {loop_parameter};

-- Step 4: Define the Property Graph
CREATE OR REPLACE PROPERTY GRAPH gXk_graph
    VERTEX TABLES (
        GXk_nodes LABEL nodes
    )
    EDGE TABLES (
        {loop_parameter} 
        SOURCE KEY (col0) REFERENCES GXk_nodes (id)
        DESTINATION KEY (col1) REFERENCES GXk_nodes (id)
        LABEL links
    );

-- Step 5: The Transitive Closure Query
SELECT * FROM GRAPH_TABLE (
    gXk_graph
    MATCH ANY (start_node:nodes)-[e:links]->{1,}(end_node:nodes)
    COLUMNS (
        start_node.id AS origin, 
        end_node.id AS destination
    )
) ORDER BY origin, destination;

Running G1k.


,origin,destination
0,0,1
1,0,2
2,0,3
3,0,4
4,0,5
...,...,...
997996,999,994
997997,999,995
997998,999,996
997999,999,997


Running G2k.


,origin,destination
0,0,1
1,0,2
2,0,3
3,0,4
4,0,5
...,...,...
3995996,1999,1994
3995997,1999,1995
3995998,1999,1996
3995999,1999,1997


Running G3k.


...

Running G4k.


,origin,destination
0,0,1
1,0,2
2,0,3
3,0,4
4,0,5
...,...,...
15991996,3999,3994
15991997,3999,3995
15991998,3999,3996
15991999,3999,3997


Running G5k.


,origin,destination
0,0,1
1,0,2
2,0,3
3,0,4
4,0,5
...,...,...
24989996,4999,4994
24989997,4999,4995
24989998,4999,4996
24989999,4999,4997


 === Timing for Transitive Closure === // Cypher //
+---------+---------------------+
| problem | time                |
+---------+---------------------+
| G1k     | 0.35087310300013996 |
| G2k     | 2.1931661440003154  |
| G3k     | 7.658980836999945   |
| G4k     | 18.23380420300009   |
| G5k     | 33.10960333700041   |
+---------+---------------------+


In [34]:
%%loop ('Distance', graphs)
%%run_duckdb

PRAGMA threads=32; -- Or however many you have
PRAGMA memory_limit='32GB'; -- Ensure it's not GC-thrashing

-- Step 2: Ensure the magic is loaded
LOAD duckpgq;

-- Step 3: Materialize the Vertices
CREATE OR REPLACE TABLE GXk_nodes AS 
    SELECT DISTINCT col0 AS id FROM {loop_parameter} 
    UNION 
    SELECT DISTINCT col1 AS id FROM {loop_parameter};

-- Step 4: Define the Property Graph
CREATE OR REPLACE PROPERTY GRAPH gXk_graph
    VERTEX TABLES (
        GXk_nodes LABEL nodes
    )
    EDGE TABLES (
        {loop_parameter} 
        SOURCE KEY (col0) REFERENCES GXk_nodes (id)
        DESTINATION KEY (col1) REFERENCES GXk_nodes (id)
        LABEL links
    );


-- Step 5: The Transitive Closure Query
SELECT * FROM GRAPH_TABLE (
    gXk_graph
    MATCH p = ANY SHORTEST (start_node:nodes)-[e:links]->{1,}(end_node:nodes)
    COLUMNS (
        start_node.id AS origin, 
        end_node.id AS destination,
        path_length(p) as distance

    )
) ORDER BY origin, destination;

Running G1k.


,origin,destination,distance
0,0,1,4
1,0,2,4
2,0,3,3
3,0,4,4
4,0,5,3
...,...,...,...
997996,999,994,4
997997,999,995,4
997998,999,996,4
997999,999,997,2


Running G2k.


,origin,destination,distance
0,0,1,4
1,0,2,4
2,0,3,4
3,0,4,4
4,0,5,3
...,...,...,...
3995996,1999,1994,4
3995997,1999,1995,4
3995998,1999,1996,3
3995999,1999,1997,4


Running G3k.
Timeout detected.
Error: Query interrupted
!!! TIMEOUT detected on G3k !!!
Running G4k.
Skipping G4k (Previous timeout).
Running G5k.
Skipping G5k (Previous timeout).
 === Timing for Distance === // Cypher //
+---------+--------------------+
| problem | time               |
+---------+--------------------+
| G1k     | 14.464675783000075 |
| G2k     | 116.94109469400018 |
| G3k     | TIMEOUT (>200s)    |
| G4k     | TIMEOUT (>200s)    |
| G5k     | TIMEOUT (>200s)    |
+---------+--------------------+


In [40]:
#trees = ['Tree5', 'Tree6']
trees = ['Tree%d' % i for i in [7, 8, 9, 10, 11, 12]]

In [41]:
%%loop ('Same Generation', trees)
%%run_duckdb

LOAD duckpgq;

-- 1. Setup Nodes
CREATE OR REPLACE TABLE GXk_nodes AS 
    SELECT DISTINCT col0 AS id FROM {loop_parameter} 
    UNION 
    SELECT DISTINCT col1 AS id FROM {loop_parameter};

-- 2. Setup Graph
CREATE OR REPLACE PROPERTY GRAPH gXk_graph
    VERTEX TABLES (
        GXk_nodes LABEL nodes
    )
    EDGE TABLES (
        {loop_parameter} 
        SOURCE KEY (col0) REFERENCES GXk_nodes (id)
        DESTINATION KEY (col1) REFERENCES GXk_nodes (id)
        LABEL links
    );

-- 3. MATERIALIZE: Save distances to a table
-- We calculate distance for ALL pairs (safest way to avoid crashes)
-- then we filter for root='0' in the next step.
CREATE OR REPLACE TABLE temp_gen_data AS 
SELECT * FROM GRAPH_TABLE (
    gXk_graph
    MATCH p = ANY SHORTEST (start_node:nodes)-[e:links]->+(end_node:nodes)
    WHERE start_node.id = '0'
    COLUMNS (
        start_node.id AS origin, 
        end_node.id AS destination,
        path_length(p) as distance
    )
);

-- 4. ANALYZE: Standard SQL Join on the saved table
-- Now we only care about paths starting from '0' (the root)
WITH root_dists AS (
    SELECT destination, distance AS gen
    FROM temp_gen_data
    WHERE origin = '0'
)
SELECT 
    r1.destination AS node_a,
    r2.destination AS node_b,
    r1.gen AS generation
FROM root_dists r1
JOIN root_dists r2 ON r1.gen = r2.gen
ORDER BY generation, node_a, node_b;

Running Tree7.


,node_a,node_b,generation
0,00,00,1
1,00,01,1
2,01,00,1
3,01,01,1
4,000,000,2
...,...,...,...
14651,0111312,0111221,6
14652,0111312,0111300,6
14653,0111312,0111310,6
14654,0111312,0111311,6


Running Tree8.


,node_a,node_b,generation
0,00,00,1
1,00,01,1
2,01,00,1
3,01,01,1
4,000,000,2
...,...,...,...
99332,01113121,01113111,7
99333,01113121,01113112,7
99334,01113121,01113113,7
99335,01113121,01113120,7


Running Tree9.


...

Running Tree10.


,node_a,node_b,generation
0,00,00,1
1,00,01,1
2,01,00,1
3,01,01,1
4,000,000,2
...,...,...,...
4153713,0111312102,0111312032,9
4153714,0111312102,0111312033,9
4153715,0111312102,0111312100,9
4153716,0111312102,0111312101,9


Running Tree11.


,node_a,node_b,generation
0,00,00,1
1,00,01,1
2,01,00,1
3,01,01,1
4,000,000,2
...,...,...,...
26253114,01113121023,01113121010,10
26253115,01113121023,01113121020,10
26253116,01113121023,01113121021,10
26253117,01113121023,01113121022,10


Running Tree12.
Timeout detected.
Error: Query interrupted
!!! TIMEOUT detected on Tree12 !!!
 === Timing for Same Generation === // Cypher //
+---------+--------------------+
| problem | time               |
+---------+--------------------+
| Tree7   | 0.0493318349999754 |
| Tree8   | 0.1653520750005555 |
| Tree9   | 0.5007336210001085 |
| Tree10  | 5.679915114000323  |
| Tree11  | 56.67059351200078  |
| Tree12  | TIMEOUT (>200s)    |
+---------+--------------------+


In [42]:
for report in reports:
    print(report)

 === Timing for Transitive Closure === // Cypher //
+---------+---------------------+
| problem | time                |
+---------+---------------------+
| G1k     | 0.35087310300013996 |
| G2k     | 2.1931661440003154  |
| G3k     | 7.658980836999945   |
| G4k     | 18.23380420300009   |
| G5k     | 33.10960333700041   |
+---------+---------------------+
 === Timing for Distance === // Cypher //
+---------+--------------------+
| problem | time               |
+---------+--------------------+
| G1k     | 14.464675783000075 |
| G2k     | 116.94109469400018 |
| G3k     | TIMEOUT (>200s)    |
| G4k     | TIMEOUT (>200s)    |
| G5k     | TIMEOUT (>200s)    |
+---------+--------------------+
 === Timing for Same Generation === // Cypher //
+---------+----------------------+
| problem | time                 |
+---------+----------------------+
| Tree5   | 0.037794283000039286 |
| Tree6   | 0.023312870000154362 |
+---------+----------------------+
 === Timing for Same Generation === // Cyphe